<a href="https://colab.research.google.com/github/yijhenjhen/pythonProject/blob/master/modelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 5s (5,899 kB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u312-b07-0ubun

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF
from pyspark.ml.linalg import Vector
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline

In [4]:
spark

In [5]:
sc

<SparkContext master=local appName=pyspark-shell>

In [6]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [7]:
data = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/oriDataTag.csv",
                      inferSchema=True,
                      header=True,
                      encoding='utf-8')

# get the dataset
def get_dataset(data):
	
    # Preprocessing and feature engineering
    feature_prep_ori = data.select('sex', 'job', 'age', 'market', 'risk', 'joinTime', 'content', 'tag').orderBy('tag')
    feature_prep = feature_prep_ori.dropna(how='any', thresh=None, subset=None)
    feature_prep = StringIndexer(inputCol="sex", outputCol="SexIndex").fit(feature_prep).transform(feature_prep)
    feature_prep = StringIndexer(inputCol="job",outputCol="jobIndex").fit(feature_prep).transform(feature_prep)
    feature_prep = StringIndexer(inputCol="joinTime",outputCol="joinTimeIndex").fit(feature_prep).transform(feature_prep)
    feature_prep = StringIndexer(inputCol="risk",outputCol="riskIndex").fit(feature_prep).transform(feature_prep)

    data = feature_prep.toPandas()
    data.to_csv('/content/drive/MyDrive/Colab Notebooks/dataTrain.csv', sep=',', encoding='utf_8_sig', index=True)
    dataTrain=data.sample(frac=0.9)
    dataTest=data[~data.index.isin(dataTrain.index)]
    dataframe = dataTrain[['SexIndex', 'jobIndex', 'age', 'riskIndex', 'joinTimeIndex', 'tag']]


    # 建立label的numpy array
    featuresArray = []
    labelArray = []
    df = dataframe['tag']
    k=0

    for i in df:
        index = (df.index)[k]
        featuresVec = [0] * 5
        featuresVec[0] = dataframe.loc[index, 'SexIndex']
        featuresVec[1] = dataframe.loc[index, 'jobIndex']
        featuresVec[2] = dataframe.loc[index, 'age']
        featuresVec[3] = dataframe.loc[index, 'riskIndex']
        featuresVec[4] = dataframe.loc[index, 'joinTimeIndex']
        featuresArray.append(np.array(featuresVec))

        tagVec = [0] * 13
        for j in i.split(','):

            try:
                j = int(j)
                if j < 14:
                    tagVec[j-1] = float(1)
                      #data.loc[index, 'tag'] = str(tagVec)
                else:
                    tagVec[j-1] = float(1)
                      #data.loc[index, 'tag'] = str(tagVec)
            
            # print(np.array(tagVec))
            except:
                pass
        labelArray.append(np.array(tagVec))
        k+=1

    featuresArray = np.array(featuresArray)
    labelArray = np.array(labelArray)


    # summarize dataset shape
    print(featuresArray.shape, labelArray.shape)
    # summarize first few examples
    for i in range(10):
        print(featuresArray[i], labelArray[i])

    return featuresArray, labelArray, dataTest

In [8]:
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(32, input_dim=n_inputs, activation='sigmoid'))# kernel_initializer='he_uniform', 
    # # model.add(Dropout(0.5))
    # model.add(Dense(128, activation='sigmoid'))# kernel_initializer='he_uniform',
    # model.add(Dense(128, activation='sigmoid'))
    # model.add(Dense(64, activation='sigmoid'))
    model.add(Dense(32, activation='sigmoid'))
    # model.add(Dense(16, activation='sigmoid'))
    # model.add(Dense(16, activation='sigmoid'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    print(model.summary())
    return model

In [ ]:
def get_model(n_inputs, n_outputs):
    tf.keras.applications.DenseNet121(
        include_top=True,
        weights="imagenet",
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
    )

In [21]:
# evaluate a model using repeated k-fold cross-validation
from sklearn.utils import class_weight
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.losses import Reduction
import torch
import matplotlib.pyplot as plt
def evaluate_model(X, y):

	results = list()
	
	time=0
	n_inputs, n_outputs = X.shape[1], y.shape[1]

	print(X.shape[1], y.shape[1])

	# define evaluation procedure
	# cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
	
	model = get_model(n_inputs, n_outputs)

	class_weight = {0:1.87, 1:4.51, 2:0.4, 3:1.8, 4:1.81, 5:0.5, 6:5,
	         7:2.99, 8:1.81, 9:8, 10:8, 11:4.2, 12:2.28}

	history = model.fit(X_train, y_train, verbose=0, epochs=2000)
	train_loss = history.history['loss']
	print(train_loss+'\n')
	epochs = range(len(train_loss))
 
	plt.figure()
	plt.plot(epochs, train_loss, color='black', label='train_loss')
	plt.ylabel("Loss (MSE)")
	plt.xlabel("Epoch")
	plt.title("Training curve")
	plt.legend(loc='upper right')
 

	yhat = model.predict(X_test)
	

	model.save("/content/drive/MyDrive/Colab Notebooks/modelTag_Sigmoid.h5")
 
 
	# loss = tf.compat.v1.losses.sigmoid_cross_entropy(
  #   y_test, yhat, weights=1.0, label_smoothing=0, scope=None,
  #   loss_collection=tf.GraphKeys.LOSSES, reduction=Reduction.SUM_BY_NONZERO_WEIGHTS)
	return results, loss

In [ ]:
# load dataset
X, y, dataset = get_dataset(data)
# evaluate model
results, loss= evaluate_model(X, y)
# summarize performance
print(loss)

(1608, 5) (1608, 13)
[ 0.  7. 46.  0.  3.] [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[ 0.  0. 40.  0.  2.] [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 0. 11. 29.  2.  2.] [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[ 0. 11. 38.  0.  3.] [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[ 0.  0. 37. 10.  1.] [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[ 0.  2. 39.  2.  0.] [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 0.  4. 30. 19.  2.] [0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.]
[ 0.  1. 26.  5.  2.] [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[ 0.  4. 40.  0.  0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[ 1. 10. 32. 16.  0.] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
5 13
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 32)                192       
                                                                 
 dense_16 (Dense)            (None, 32)                105

In [18]:
print(type(loss))

<class 'tensorflow.python.framework.ops.EagerTensor'>


In [15]:
from tensorflow.keras.models import load_model
import random
pd.options.display.max_columns = None
test = list()


X, y, dataset = get_dataset(data)
dataset.reset_index(inplace=True, drop=False)
totalACC=0
for epo in range(10):

    for t in range(178):
        x = random.randrange(0, 178)
        test.append(x)


    dataframe = dataset[['SexIndex', 'jobIndex', 'age', 'riskIndex', 'joinTimeIndex', 'tag']]

    model = load_model('/content/drive/MyDrive/Colab Notebooks/modelTag_V3.h5',compile=False)
    acc=0
    for i in test:
        # print(i)

        index = (dataset['index'].index)[0]
        featuresVec = [0] * 5
        featuresVec[0] = dataset.loc[i, 'SexIndex']
        featuresVec[1] = dataset.loc[i, 'jobIndex']
        featuresVec[2] = dataset.loc[i, 'age']
        featuresVec[3] = dataset.loc[i, 'riskIndex']
        featuresVec[4] = dataset.loc[i, 'joinTimeIndex']

        input = np.array(featuresVec).reshape((1,5))
        # print(input)
        y_predicted = model.predict(input)
        # print(y_predicted.round(4))
        # print(dataset[i:i+1])
        

        resh = y_predicted[0]
        top_k = 3
        top_k_idx = resh.argsort()[::-1][0:top_k]
        top_k_idx += 1
        print("困擾類型:", top_k_idx, dataset.loc[i, 'tag'])
        right=0

        nums=dataset.loc[i, 'tag'].split(',')
        for k in nums:
          if int(str(k)) in top_k_idx:
            right+=1
          else:
            pass
        correct = right/len(nums)
        # print(correct)
        acc+=correct
        # print('####################################################################')
    accCache = acc/len(test)
    totalACC += accCache               
    print(epo)



print('TotalACC:', totalACC/epo)

串流輸出內容已截斷至最後 5000 行。
困擾類型: [ 1  6 12] 6
困擾類型: [3 4 6] 6,4,3
困擾類型: [ 3 13  9] 3
困擾類型: [ 4  3 13] 4,3
困擾類型: [ 3 13  9] 4,3
困擾類型: [3 9 1] 3
困擾類型: [13  6  3] 1
困擾類型: [5 9 3] 5
困擾類型: [ 3 13  9] 3
困擾類型: [3 6 9] 8
困擾類型: [12  1  9] 9
困擾類型: [ 1 13  2] 6,1
困擾類型: [3 9 7] 3
困擾類型: [ 6  9 13] 6
困擾類型: [ 1  5 13] 3
困擾類型: [ 6  5 13] 6
困擾類型: [11 12 13] 6
困擾類型: [13  1  3] 13
困擾類型: [ 3 11  6] 13
困擾類型: [3 1 4] 8,9
困擾類型: [2 3 4] 6
困擾類型: [ 3 13  9] 3
困擾類型: [ 1 13  2] 6,1
困擾類型: [ 1 13  2] 6,1
困擾類型: [3 4 5] 6,3,4
困擾類型: [3 4 9] 3,4
困擾類型: [ 2 13  3] 13,2
困擾類型: [3 6 4] 6,3
困擾類型: [3 1 4] 1
困擾類型: [6 9 2] 6,9
困擾類型: [6 3 1] 1,13
困擾類型: [ 4  3 10] 1
困擾類型: [5 3 4] 5
困擾類型: [12  1  9] 9
困擾類型: [13  6  9] 13
困擾類型: [ 2  4 13] 4,13
困擾類型: [4 2 6] 2,9
困擾類型: [13  6  3] 13,1
困擾類型: [ 6 12  9] 6
困擾類型: [ 6  2 13] 6
困擾類型: [9 6 2] 13,3
困擾類型: [ 6 13  4] 6
困擾類型: [ 3  5 13] 6
困擾類型: [ 3 13  9] 4,3
困擾類型: [6 8 4] 1,8
困擾類型: [ 9 10 13] 2
困擾類型: [ 9 12  2] 10
困擾類型: [6 4 8] 1,9
困擾類型: [3 9 4] 6
困擾類型: [ 3 13  9] 3
困擾類型: [12  4 11] 6
困擾類型: [ 3  4 1

In [ ]:
import tensorflow as tf
tf.__version__